In [13]:
import psycopg2
from psycopg2 import sql
import psycopg2.extensions
import numpy as np

def adapt_numpy_int64(numpy_int64):
    return psycopg2.extensions.AsIs(int(numpy_int64))

psycopg2.extensions.register_adapter(np.int64, adapt_numpy_int64)

class DatabaseController:
    def __init__(self, dbname="meta_learning", user="user", password="1234", host="localhost", port="5432"):
        self.db_params = {
            "dbname": dbname,
            "user": user,
            "password": password,
            "host": host,
            "port": port
        }
        self.conn = None
        self.cursor = None

    def _get_connection(self):
        if not self.conn:
            self.conn = psycopg2.connect(**self.db_params)
            self.cursor = self.conn.cursor()
        return self.conn, self.cursor
    
    def close_connection(self):
        if self.conn:
            self.conn.commit()
            self.cursor.close()
            self.conn.close()
            self.conn = None
            self.cursor = None

    def execute_query(self, query, params=None, fetch=False):
        conn, cursor = self._get_connection()
        try:
            cursor.execute(query, params)
            if fetch:
                return cursor.fetchall()
            conn.commit()
        except Exception as e:
            conn.rollback()
            print("Error:", e)
            raise e

    def create_table(self, table_name, columns):
        columns_str = ", ".join([f"{col} {dtype}" for col, dtype in columns.items()])
        query = sql.SQL(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_str});")
        self.execute_query(query)

    def insert_data(self, table_name, data):
        values_placeholders = ", ".join(["%s"] * len(data))
        query = sql.SQL(f"INSERT INTO {table_name} VALUES ({values_placeholders});")
        self.execute_query(query, tuple(data.values()))

    def fetch_data(self, table_name, conditions=None):
        base_query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(table_name))
        
        if conditions:
            where_clause = sql.SQL(" WHERE ") + sql.SQL(" AND ").join(
                sql.Composed([sql.Identifier(col), sql.SQL("= %s")]) for col in conditions.keys()
            )
            query = base_query + where_clause
            return self.execute_query(query, tuple(conditions.values()), fetch=True)
        
        return self.execute_query(base_query, fetch=True)

    def clear_table(self, table_name):
        query = sql.SQL(f"DELETE FROM {table_name};")
        self.execute_query(query)

    def get_all_data(self, table_name):
        query = sql.SQL(f"SELECT * FROM {table_name};")
        return self.execute_query(query, fetch=True)

    def get_data_length(self, table_name):
        query = sql.SQL(f"SELECT COUNT(*) FROM {table_name};")
        result = self.execute_query(query, fetch=True)
        return result[0][0] if result else 0
    
    def table_exists(self, table_name):
        query = sql.SQL("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = %s);")
        result = self.execute_query(query, (table_name,), fetch=True)
        return result[0][0] if result else False
    
    def get_columns(self, table_name):
        query = sql.SQL("SELECT column_name FROM information_schema.columns WHERE table_name = %s;")
        result = self.execute_query(query, (table_name,), fetch=True)
        return [row[0] for row in result] if result else []
    
    def delete_by_condition(self, table_name, conditions):    
        where_clause = " AND ".join([f"{col} = %s" for col in conditions.keys()])
        query = sql.SQL(f"DELETE FROM {table_name} WHERE {where_clause};")
        self.execute_query(query, tuple(conditions.values()))

    def update_data(self, table_name, updates, conditions):
        set_clause = ", ".join([f"{col} = %s" for col in updates.keys()])
        where_clause = " AND ".join([f"{col} = %s" for col in conditions.keys()])
        query = sql.SQL(f"UPDATE {table_name} SET {set_clause} WHERE {where_clause};")
        self.execute_query(query, tuple(updates.values()) + tuple(conditions.values()))
    
    def get_tables(self):
        query = sql.SQL("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
        result = self.execute_query(query, fetch=True)
        return [row[0] for row in result] if result else []

    def drop_all_tables(self):
        query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        AND table_type = 'BASE TABLE';
        """
        tables = self.execute_query(query, fetch=True)

        for table in tables:
            table_name = table[0]
            query = sql.SQL(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
            self.execute_query(query)

        print("All tables have been dropped.")


    def execute_raw_query(self, query, params=None):
        return self.execute_query(query, params, fetch=True)



In [14]:
db = DatabaseController()
db.drop_all_tables()

db.create_table("images", {
    "id": "INTEGER PRIMARY KEY",
    "image_name": "TEXT NOT NULL UNIQUE"
})

db.create_table("classes", {
    "id": "INTEGER PRIMARY KEY",
    "class_name": "TEXT NOT NULL UNIQUE"
})

db.create_table("image_class_labels", {
    "image_id": "INTEGER NOT NULL",
    "class_id": "INTEGER NOT NULL",    
    "PRIMARY KEY (image_id)": "",
    "FOREIGN KEY (image_id)": "REFERENCES images(id) ON DELETE CASCADE",
    "FOREIGN KEY (class_id)": "REFERENCES classes(id) ON DELETE CASCADE"
    })

db.create_table("train_test_split", {
    "image_id": "INTEGER NOT NULL PRIMARY KEY",
    "is_training_image": "INTEGER NOT NULL CHECK(is_training_image IN (0, 1))",
    "FOREIGN KEY (image_id)": "REFERENCES images(id) ON DELETE CASCADE"
    })

# db.create_table("bounding_boxes", {
#     "image_id": "INTEGER NOT NULL PRIMARY KEY",
#     "x": "REAL NOT NULL",
#     "y": "REAL NOT NULL",
#     "width": "REAL NOT NULL",
#     "height": "REAL NOT NULL",
#     "FOREIGN KEY (image_id)": "REFERENCES images(id) ON DELETE CASCADE"
#     })

# db.create_table("parts", {
#     "id": "INTEGER PRIMARY KEY",
#     "part_name": "TEXT NOT NULL UNIQUE"
# })

# db.create_table("part_locs", {
#     "id": "INTEGER PRIMARY KEY",
#     "image_id": "INTEGER NOT NULL",
#     "part_id": "INTEGER NOT NULL",
#     "x": "REAL NOT NULL",
#     "y": "REAL NOT NULL",
#     "visible": "INTEGER NOT NULL CHECK(visible IN (0, 1))",
#     "FOREIGN KEY (image_id)": "REFERENCES images(id) ON DELETE CASCADE",
#     "FOREIGN KEY (part_id)": "REFERENCES parts(id) ON DELETE CASCADE",
#     "UNIQUE (image_id, part_id)": ""
#     })

# db.create_table("attributes", {
#     "id": "INTEGER PRIMARY KEY",
#     "attribute_name": "TEXT NOT NULL UNIQUE"
# })

# db.create_table("certainties", {
#     "id": "INTEGER PRIMARY KEY",
#     "certainty_name": "TEXT NOT NULL UNIQUE"
# })

# db.create_table("image_attribute_labels", {
#     "id": "INTEGER PRIMARY KEY",
#     "image_id": "INTEGER NOT NULL",
#     "attribute_id": "INTEGER NOT NULL",
#     "is_present": "INTEGER NOT NULL CHECK(is_present IN (0, 1))",
#     "certainty_id": "INTEGER NOT NULL",
#     "time": "REAL NOT NULL",
#     "FOREIGN KEY (image_id)": "REFERENCES images(id) ON DELETE CASCADE",
#     "FOREIGN KEY (attribute_id)": "REFERENCES attributes(id) ON DELETE CASCADE",
#     "FOREIGN KEY (certainty_id)": "REFERENCES certainties(id) ON DELETE CASCADE"
# })

# db.create_table("class_attribute_labels_continuous", {
#     "id": "INTEGER PRIMARY KEY",
#     "class_id": "INTEGER NOT NULL",
#     "attribute_id": "INTEGER NOT NULL",
#     "value": "REAL NOT NULL",
#     "FOREIGN KEY (class_id)": "REFERENCES classes(id) ON DELETE CASCADE",
#     "FOREIGN KEY (attribute_id)": "REFERENCES attributes(id) ON DELETE CASCADE",
#     "UNIQUE (class_id, attribute_id)": ""
# })

# table_definition = {
#     "id": "INTEGER PRIMARY KEY",
#     "class_id": "INTEGER NOT NULL"
# }

# # Dodaj 312 atrybutów
# for i in range(1, 313):
#     table_definition[f"attribute_{i}"] = f"REAL NOT NULL CHECK(attribute_{i} >= 0 AND attribute_{i} <= 100)"

# # Dodaj ograniczenia
# table_definition["FOREIGN KEY (class_id)"] = "REFERENCES classes(id) ON DELETE CASCADE"
# table_definition["CONSTRAINT uq_class_attr UNIQUE (class_id)"] = ""

# db.create_table("class_attribute_labels_continuous", table_definition)

# db.create_table("part_click_locs", {
#     "id": "INTEGER PRIMARY KEY",
#     "image_id": "INTEGER NOT NULL",
#     "part_id": "INTEGER NOT NULL",
#     "x": "REAL NOT NULL",
#     "y": "REAL NOT NULL",
#     "visible": "INTEGER NOT NULL CHECK(visible IN (0, 1))",
#     "time": "REAL NOT NULL",
#     "FOREIGN KEY (image_id)": "REFERENCES images(id) ON DELETE CASCADE",
#     "FOREIGN KEY (part_id)": "REFERENCES parts(id) ON DELETE CASCADE"
# })
db.get_tables()

All tables have been dropped.


['images', 'image_class_labels', 'classes', 'train_test_split']

In [15]:
import os 
import pandas as pd

data_dir = os.path.join(os.getcwd(), 'data/raw/CUB_200_2011')

def load_data_from_txt(file_name, delimiter=' ', dtype=None):
    file_path = os.path.join(data_dir, file_name)
    df = pd.read_csv(file_path, sep=delimiter, header=None, dtype=dtype)
    return df

def load_data_from_txt_with_id(file_name, delimiter=' ', dtype=None):
    file_path = os.path.join(data_dir, file_name)
    df = pd.read_csv(file_path, sep=delimiter, header=None, dtype=dtype)
    df.insert(0, 'id', range(1, len(df) + 1))

    df.columns = [str(i) for i in range(len(df.columns))]

    return df

def insert_data_from_dataframe(data, table_name):
    for index, row in data.iterrows():
        data = {f"col{i}": row[i] for i in range(len(row))}
        db.insert_data(table_name, data)

images_file = os.path.join(data_dir, 'images.txt')
classes_file = os.path.join(data_dir, 'classes.txt')
image_class_labels_file = os.path.join(data_dir, 'image_class_labels.txt')
train_test_split_file = os.path.join(data_dir, 'train_test_split.txt')
# bounding_boxes_file = os.path.join(data_dir, 'bounding_boxes.txt')
# parts_file = os.path.join(data_dir, 'parts/parts.txt')
# part_locs_file = os.path.join(data_dir, 'parts/part_locs.txt')
# part_click_locs_file = os.path.join(data_dir, 'parts/part_click_locs.txt')
# attributes_file = os.path.join(data_dir, 'attributes/attributes.txt')
# certainties_file = os.path.join(data_dir, 'attributes/certainties.txt')
# image_attribute_labels_file = os.path.join(data_dir, 'attributes/image_attribute_labels.txt')
# class_attribute_labels_continuous_file = os.path.join(data_dir, 'attributes/class_attribute_labels_continuous.txt')

image_data = load_data_from_txt(images_file, delimiter=' ', dtype={0: int, 1: str})
classes_data = load_data_from_txt(classes_file, delimiter=' ', dtype={0: int, 1: str})
image_class_labels_data = load_data_from_txt(image_class_labels_file, delimiter=' ', dtype={0: int, 1: int})
train_test_split_data = load_data_from_txt(train_test_split_file, delimiter=' ', dtype={0: int, 1: int})
# bounding_boxes_data = load_data_from_txt(bounding_boxes_file, delimiter=' ', dtype={0: int, 1: int, 2: int, 3: int, 4: int})
# parts_data = load_data_from_txt(parts_file, delimiter=' ', dtype={0: int, 1: str})
# part_locs_data = load_data_from_txt_with_id(part_locs_file, delimiter=' ', dtype={0: int, 1: int, 2: int, 3: int, 4: int, 5: int})
# part_click_locs_data = load_data_from_txt_with_id(part_click_locs_file, delimiter=' ',  dtype={0: int, 1: int, 2: int, 3: int, 4: int, 5: float})
# attributes_data = load_data_from_txt(attributes_file, delimiter=' ', dtype={0: int, 1: str})
# certainties_data = load_data_from_txt(certainties_file, delimiter=' ', dtype={0: int, 1: str})
# image_attribute_labels_data = load_data_from_txt_with_id(image_attribute_labels_file, delimiter=' ', dtype={0: int, 1: int, 2: int, 3: int, 4: float})
# class_attribute_labels_continuous_data = load_data_from_txt(class_attribute_labels_continuous_file, delimiter=' ')

insert_data_from_dataframe(image_data, "images")
insert_data_from_dataframe(classes_data, "classes")
insert_data_from_dataframe(image_class_labels_data, "image_class_labels")
insert_data_from_dataframe(train_test_split_data, "train_test_split")
# insert_data_from_dataframe(bounding_boxes_data, "bounding_boxes")
# insert_data_from_dataframe(parts_data, "parts")
# insert_data_from_dataframe(part_locs_data, "part_locs")
# insert_data_from_dataframe(part_click_locs_data, "part_click_locs")
# insert_data_from_dataframe(attributes_data, "attributes")
# insert_data_from_dataframe(certainties_data, "certainties")
# insert_data_from_dataframe(image_attribute_labels_data, "image_attribute_labels")
#insert_data_from_dataframe(class_attribute_labels_continuous_data, "class_attribute_labels_continuous")

0     int64
1    object
dtype: object
0                                                    1
1    001.Black_footed_Albatross/Black_Footed_Albatr...
Name: 0, dtype: object
0     int64
1    object
dtype: object
0                             1
1    001.Black_footed_Albatross
Name: 0, dtype: object
0    int64
1    int64
dtype: object
0    1
1    1
Name: 0, dtype: int64
0    int64
1    int64
dtype: object
0    1
1    0
Name: 0, dtype: int64


In [18]:
class DataLoader:
    def __init__(self):
        self.db = DatabaseController()
    def fetch_table_as_df(self, table_name):
        data = self.db.get_all_data(table_name)
        columns = self.db.get_columns(table_name)
        return pd.DataFrame(data, columns=columns) if data else pd.DataFrame(columns=columns)
    def load_and_merge_data(self):
        df_images = self.fetch_table_as_df("images")
        df_classes = self.fetch_table_as_df("classes")
        df_labels = self.fetch_table_as_df("image_class_labels")
        df_split = self.fetch_table_as_df("train_test_split")

        if df_images.empty or df_classes.empty or df_labels.empty or df_split.empty:
            print("⚠️ Data missing from one of the tables!")
            return None
        
        df = df_images.merge(df_labels, left_on="id", right_on="image_id")
        df = df.merge(df_classes, left_on="class_id", right_on="id", suffixes=("", "_class"))
        df = df.merge(df_split, on="image_id")

        df.drop(columns=["id_class"], inplace=True, errors="ignore")

        return df

data_loader = DataLoader()
df_combined = data_loader.load_and_merge_data()

print(df_combined.head())

   id                                         image_name  image_id  class_id  \
0   1  001.Black_footed_Albatross/Black_Footed_Albatr...         1         1   
1   2  001.Black_footed_Albatross/Black_Footed_Albatr...         2         1   
2   3  001.Black_footed_Albatross/Black_Footed_Albatr...         3         1   
3   4  001.Black_footed_Albatross/Black_Footed_Albatr...         4         1   
4   5  001.Black_footed_Albatross/Black_Footed_Albatr...         5         1   

                   class_name  is_training_image  
0  001.Black_footed_Albatross                  0  
1  001.Black_footed_Albatross                  1  
2  001.Black_footed_Albatross                  0  
3  001.Black_footed_Albatross                  1  
4  001.Black_footed_Albatross                  1  
